In [1]:
from bs4 import BeautifulSoup
import requests
from urllib.request import Request, urlopen
import time
import pandas as pd

In [2]:
fiis_list = ["ALZR11", "BRCR11", "BTLG11", "CPTS11", "HGLG11", "HGRE11", "HSML11", "KNCR11", "KNRI11", "MXRF11", "VISC11", "XPLG11", "XPML11"]


In [3]:
fiis_indicators = []
for fii in fiis_list:
  url = f"https://statusinvest.com.br/fundos-imobiliarios/{fii}"
  
  time.sleep(0.25)
  try:
    req = Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Mobile Safari/537.36')
    html = urlopen(req)
    soup = BeautifulSoup(html, 'html.parser')
      
    fund = soup.find("div", id="fund-section")
    main = soup.find("main", id="main-2")

    name = fund.find_all("div")[16].find("strong").getText()
    init = fund.find_all("div")[19].find("strong").getText()
    sector = fund.find_all("div")[28].find("strong").getText()
    type = fund.find_all("div")[26].find("strong").getText()
    administrator = fund.find("div", "container").find_all("div", recursive=False)[2].find("strong", "fw-700").getText()

    price = main.find("div", title="Valor atual do ativo").find("strong").getText()
    dividend_yield = main.find("div", title="Dividend Yield com base nos últimos 12 meses").find("strong").getText()
    net_worth = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[0].find("span", "sub-value").getText().replace("R$ ","")
    asset_value = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[0].find("strong").getText()
    pvp = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[1].find("strong").getText()
    shareholders = main.find("div", "top-info top-info-2 top-info-md-3 top-info-lg-n d-flex justify-between").find_all("div", recursive=False)[5].find("strong").getText()
    lastDiv = main.find("div", "mt-5 d-flex flex-wrap flex-lg-nowrap justify-between").find_all("div", recursive=False)[1].find("strong").getText()
    nextDiv = main.find("div", "mt-5 d-flex flex-wrap flex-lg-nowrap justify-between").find_all("div", recursive=False)[2].find("strong").getText()

    dict = {  "fii": fii,
        "nome": name,
        "segmento": sector,
        "tipo": type,
        "administrador": administrator,
        "cotistas" : shareholders,
        "patrimonio": net_worth,
        "vp" : asset_value,
        "valor_atual": price,
        "pvp": pvp,
        "dividend_yield": dividend_yield,
        "ultimo dividendo" : lastDiv,
        "proximo dividendo" : nextDiv,
    }
    fiis_indicators.append(dict)
  except Exception as e:
    print(e)

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


In [4]:
print(fiis_indicators)

[{'fii': 'ALZR11', 'nome': 'Alianza Trust Renda Imobiliária', 'segmento': 'Híbrido', 'tipo': 'Renda', 'administrador': 'BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM', 'cotistas': '147.506', 'patrimonio': '1.059.348.545', 'vp': '107,43', 'valor_atual': '113,87', 'pvp': '1,06', 'dividend_yield': '8,52', 'ultimo dividendo': '0,8250', 'proximo dividendo': '0,8250'}, {'fii': 'BRCR11', 'nome': 'BTG Pactual Corporate Office', 'segmento': 'Híbrido', 'tipo': 'Híbrido', 'administrador': 'BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM', 'cotistas': '152.081', 'patrimonio': '2.671.875.040', 'vp': '100,30', 'valor_atual': '57,88', 'pvp': '0,58', 'dividend_yield': '9,31', 'ultimo dividendo': '0,4100', 'proximo dividendo': '-'}, {'fii': 'BTLG11', 'nome': 'FII BTLG', 'segmento': 'Híbrido', 'tipo': 'Renda', 'administrador': 'BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM', 'cotistas': '254.883', 'patrimonio': '2.822.465.817', 'vp': '99,06', 'valor_atual': '99,53', 'pvp': '1,00', 'dividend_yield': '9,02', 'ultimo di

In [5]:
df = pd.DataFrame.from_dict(fiis_indicators)

In [6]:
df.to_excel("fiis.xlsx", index=False)